In [28]:
import re 
from collections import defaultdict
from helpers     import data 

In [6]:
rules = data(7)
rules[:5]

['shiny purple bags contain 2 pale blue bags, 1 wavy fuchsia bag, 5 pale salmon bags.',
 'bright gray bags contain 4 dotted coral bags.',
 'clear chartreuse bags contain 3 dark magenta bags, 3 dull gray bags, 4 dark silver bags.',
 'posh maroon bags contain 5 bright brown bags, 3 posh brown bags, 4 clear bronze bags.',
 'wavy plum bags contain 2 dull turquoise bags, 2 dotted yellow bags, 2 drab silver bags, 5 wavy violet bags.']

**Part 1:** Count the number of bags that can ultimately contain a gold bag. It would be best if we could change `rules` to map `contained_bag` to a dictionary of `container_bags`. 

In [48]:
contained_to_container_map = defaultdict(list)
for rule in rules: 
    container_rule, contained_rules = rule.split("contain")
    contained_rules = contained_rules.split(",")
    
    bag_index = re.search("bag", container_rule).start()
    container_color = container_rule[:bag_index]

    for contained_rule in contained_rules: 
        # Remove number and whitespaces 
        char_index = re.search("[a-z]", contained_rule).start() 
        contained_rule = contained_rule[char_index:]
        bag_index = re.search("bag", contained_rule).start()
        contained_color = contained_rule[:bag_index]
        contained_rule = contained_rule.strip()
        contained_to_container_map[contained_color].append(container_color)

In [49]:
contained_to_container_map

defaultdict(list,
            {'pale blue ': ['shiny purple ',
              'wavy violet ',
              'shiny plum ',
              'clear lavender ',
              'plaid bronze ',
              'light green '],
             'wavy fuchsia ': ['shiny purple ',
              'faded yellow ',
              'bright turquoise ',
              'bright lavender '],
             'pale salmon ': ['shiny purple ',
              'wavy violet ',
              'vibrant maroon ',
              'faded tomato ',
              'posh white ',
              'dark turquoise '],
             'dotted coral ': ['bright gray ',
              'wavy lime ',
              'dark teal ',
              'drab crimson '],
             'dark magenta ': ['clear chartreuse ',
              'clear green ',
              'shiny yellow ',
              'muted green ',
              'dull coral ',
              'light white ',
              'mirrored purple '],
             'dull gray ': ['clear chartreuse ',
         